In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

In [2]:
model = LinearDiscriminantAnalysis
coin_names = ['Bitcoin', 'Litecoin','Ethereum', 'Dogecoin', 'XRP']

In [27]:
coin_name = 'Bitcoin'
twitter_signals = pd.read_csv('{0}_twitter_signals.csv'.format(coin_name))
twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: x.strftime('%m-%d-%Y'))
news_signals = pd.read_csv('{0}_news_signals.csv'.format(coin_name))
news_signals.drop(columns = ['compound','positive','neutral','negative','polarity','subjective'], inplace=True)
# twitter_signals.drop(columns = ['label','Returns','Adj Close'], inplace=True)
combined_signals = twitter_signals.merge(news_signals, left_on='Date', right_on='Date')
combined_signals

,Date,compound,positive,neutral,negative,polarity,subjective,Adj Close,Returns,label
0,03-17-2022,0.257617,0.114795,0.853584,0.031627,0.164160,0.292578,40951.378906,-0.004680,0
1,03-18-2022,0.214339,0.112564,0.850381,0.037047,0.150200,0.328900,41801.156250,0.020751,0
2,03-19-2022,0.235347,0.103331,0.877785,0.018882,0.166291,0.314085,42190.652344,0.009318,0
3,03-20-2022,0.239058,0.117261,0.830967,0.051772,0.131462,0.296382,41247.824219,-0.022347,0
4,03-21-2022,0.278229,0.133310,0.844858,0.021834,0.181888,0.325414,41077.996094,-0.004117,0
5,03-22-2022,0.239663,0.101343,0.881325,0.017332,0.141516,0.330665,42358.808594,0.031180,1
6,03-23-2022,0.251125,0.110229,0.863092,0.026673,0.158174,0.330588,42892.957031,0.012610,0
7,03-24-2022,0.213862,0.100086,0.869181,0.030706,0.153200,0.338225,43960.933594,0.024899,0
8,03-25-2022,0.102883,0.078818,0.877994,0.043191,0.113496,0.426744,44348.730469,0.008821,0
9,03-26-2022,0.242448,0.112143,0.843863,0.044019,0.139350,0.344334,44500.828125,0.003430,0


# train individually

In [32]:
for coin_name in coin_names:
    twitter_signals = pd.read_csv('{0}_twitter_signals.csv'.format(coin_name))
    twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: x.strftime('%m-%d-%Y'))
    news_signals = pd.read_csv('{0}_news_signals.csv'.format(coin_name))
    news_signals.drop(columns = ['compound','positive','neutral','negative','polarity','subjective'], inplace=True)
    # twitter_signals.drop(columns = ['label','Returns','Adj Close'], inplace=True)
    combined_signals = twitter_signals.merge(news_signals, left_on='Date', right_on='Date')
    # combined_signals = news_signals.append(twitter_signals, ignore_index=True)
    X = combined_signals.drop(columns = ['Date', 'label','Returns','Adj Close'])
    y = combined_signals['label']
    classifier = model()
    classifier.fit(X,y)
    pickle.dump(classifier, open(coin_name+'_twitter.sav', 'wb'))

In [33]:
loaded_model = pickle.load(open('XRP_twitter.sav', 'rb'))
loaded_model.score(X, y)

0.8

In [34]:
[(i,j) for i, j in zip(y,loaded_model.predict(X))]

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (-1, -1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (-1, 0),
 (1, 0),
 (-1, -1),
 (0, 0),
 (0, 0),
 (-1, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, -1)]

In [35]:
for coin_name in coin_names:
    # twitter_signals = pd.read_csv('{0}_twitter_signals.csv'.format(coin_name))
    # twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: x.strftime('%m-%d-%Y'))
    news_signals = pd.read_csv('{0}_news_signals.csv'.format(coin_name))
    # news_signals.drop(columns = ['compound','positive','neutral','negative','polarity','subjective'], inplace=True)
    # twitter_signals.drop(columns = ['label','Returns','Adj Close'], inplace=True)
    combined_signals = news_signals
    # combined_signals = news_signals.append(twitter_signals, ignore_index=True)
    X = combined_signals.drop(columns = ['Date', 'label','Returns','Adj Close'])
    y = combined_signals['label']
    classifier = model()
    classifier.fit(X,y)
    pickle.dump(classifier, open(coin_name+'_news.sav', 'wb'))

In [36]:
loaded_model = pickle.load(open('XRP_news.sav', 'rb'))
loaded_model.score(X, y)

0.8666666666666667

In [37]:
[(i,j) for i, j in zip(y,loaded_model.predict(X))]

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (-1, -1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (-1, -1),
 (1, 0),
 (-1, 0),
 (0, 0),
 (0, 0),
 (-1, -1),
 (1, 0),
 (0, 0),
 (0, 0),
 (1, 1)]

# concatenating

In [38]:
for coin_name in coin_names:
    twitter_signals = pd.read_csv('{0}_twitter_signals.csv'.format(coin_name))
    twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: x.strftime('%m-%d-%Y'))
    news_signals = pd.read_csv('{0}_news_signals.csv'.format(coin_name))
    # news_signals.drop(columns = ['compound','positive','neutral','negative','polarity','subjective'], inplace=True)
    # twitter_signals.drop(columns = ['label','Returns','Adj Close'], inplace=True)
    combined_signals = twitter_signals.merge(news_signals, left_on='Date', right_on='Date')
    # combined_signals = news_signals.append(twitter_signals, ignore_index=True)
    X = combined_signals.drop(columns = ['Date', 'label','Returns','Adj Close'])
    y = combined_signals['label']
    classifier = model()
    classifier.fit(X,y)
    pickle.dump(classifier, open(coin_name+'_concat.sav', 'wb'))

In [39]:
loaded_model = pickle.load(open('XRP_concat.sav', 'rb'))
loaded_model.score(X, y)

0.8666666666666667

In [40]:
[(i,j) for i, j in zip(y,loaded_model.predict(X))]

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (-1, -1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (-1, -1),
 (1, 0),
 (-1, -1),
 (0, 0),
 (0, 0),
 (-1, -1),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, -1)]